## Multi-Agent Journalism Workshop: Collaborative AI Article Writing

Welcome to this interactive workshop on multi-agent AI systems! In this notebook, we'll explore how multiple AI agents can work together to write, edit, and fact-check a news article - similar to how a real newsroom operates.

### Workshop Overview

This demo showcases:
1. 🤖 **Multiple AI Agents** working together, each with a specific role
2. 🔍 **Real-time fact checking** using Bing Search
3. ✍️ **Iterative improvement** of content through collaboration

### The Agents

Our system uses four specialized agents:
- **Writer Agent** 📝 - Creates and revises the article
- **Editor Agent** 📋 - Reviews and suggests improvements
- **Verifier Agent** ✔️ - Fact-checks using Bing Search
- **Orchestrator Agent** 🎯 - Manages the workflow and ensures quality

Let's see how they work together!

### Setup

First, let's import the necessary libraries and set up our environment. We'll need:
- Azure AI Project libraries for creating and managing our agents
- Rich library for beautiful console output
- Environment variables for secure configuration

In [ ]:
import datetime
import json
import os
from dotenv import load_dotenv
from typing import List
from rich.console import Console
from rich.text import Text
from rich.markdown import Markdown

# Azure AI Project imports
from azure.ai.projects import AIProjectClient
from azure.ai.projects.models import MessageRole, BingGroundingTool
from azure.identity import DefaultAzureCredential

# Initialize Rich console for pretty output
console = Console()

# Load environment variables
load_dotenv()

# Set up Azure AI Project variables
os.environ["PROJECT_CONNECTION_STRING"] = os.getenv("PROJECT_CONNECTION_STRING", "")
os.environ["MODEL_DEPLOYMENT_NAME"] = os.getenv("MODEL_DEPLOYMENT_NAME", "")
os.environ["BING_CONNECTION_NAME"] = os.getenv("BING_CONNECTION_NAME", "")


### Helper Functions

Let's define a helper function to display agent responses in a clear, formatted way. This will make it easier to follow the conversation between agents.


In [ ]:
def print_agent_response(project_client, thread_id, agent_role):
    """
    Prints an agent's response with proper formatting and citation links.
    
    Args:
        project_client: The AI Project client instance
        thread_id: The ID of the current conversation thread
        agent_role: The role of the agent (e.g., "Writer", "Editor")
    """
    response_message = project_client.agents.list_messages(thread_id=thread_id).get_last_message_by_role(
        MessageRole.AGENT
    )
    if response_message:
        console.print(f"\n[bold magenta]{agent_role}[/bold magenta]: ")
        for text_message in response_message.text_messages:
            console.print(Markdown(text_message.text.value))
        for annotation in response_message.url_citation_annotations:
            console.print(f"[italic]Source: [{annotation.url_citation.title}]({annotation.url_citation.url})[/italic]")
    else:
        console.print(f"[bold red]No response message found for {agent_role}[/bold red]")


In [ ]:
def create_journalism_research_agent():
    """
    Creates a team of AI agents for collaborative article writing.
    Returns the project client and all agent instances.
    """
    # Initialize the AI Project client
    project_client = AIProjectClient.from_connection_string(
        credential=DefaultAzureCredential(),
        conn_str=os.environ["PROJECT_CONNECTION_STRING"],
    )

    # Set up Bing Search tool
    bing_connection = project_client.connections.get(connection_name=os.environ["BING_CONNECTION_NAME"])
    bing = BingGroundingTool(connection_id=bing_connection.id)
    console.print("✨ Initialized Bing Search tool for research", style="green")

    # Create Writer Agent
    writer_agent = project_client.agents.create_agent(
        model=os.environ["MODEL_DEPLOYMENT_NAME"],
        name="writer_assistant",
        instructions="""You are a high-quality journalist agent who excels at writing a first draft 
        of an article as well as revising the article based on feedback from the other agents. 
        Do not just write bullet points on how you would write the article, but actually write it. 
        Don't ask if it's necessary to revise the article when asked, just write the article with the recommendations.""",
        tools=bing.definitions,
        headers={"x-ms-enable-preview": "true"},
    )
    console.print("📝 Created Writer Agent", style="green")

    # Create Editor Agent
    editor_agent = project_client.agents.create_agent(
        model=os.environ["MODEL_DEPLOYMENT_NAME"],
        name="editor_agent",
        instructions="""You are an expert editor. You carefully read an article and make suggestions 
        for improvements and suggest additional topics that should be researched to improve the article quality.""",
        tools=bing.definitions,
        headers={"x-ms-enable-preview": "true"},
    )
    console.print("📋 Created Editor Agent", style="green")

    # Create Verifier Agent
    verifier_agent = project_client.agents.create_agent(
        model=os.environ["MODEL_DEPLOYMENT_NAME"],
        name="verifier_agent",
        instructions="""You are responsible for ensuring the article's accuracy. You should use the 
        Bing tool to search the internet to verify any relevant facts. Ask the writer for rewrites 
        if you find inaccuracies. Cite your sources and pass back to writer for inclusion in the final article.""",
        tools=bing.definitions,
        headers={"x-ms-enable-preview": "true"},
    )
    console.print("✔️ Created Verifier Agent", style="green")

    # Create Orchestrator Agent
    orchestrator_agent = project_client.agents.create_agent(
        model=os.environ["MODEL_DEPLOYMENT_NAME"],
        name="orchestrator_agent",
        instructions="""You are leading a journalism team that conducts research to craft high-quality 
        journalist articles. You ensure that the output contains an actual well-written article, not just 
        bullet points on what or how to write the article. If the article isn't to that level yet, ask 
        the writer for a rewrite. If the team has written a strong article with a clear point that meets 
        the requirements, and has been reviewed by the editor, and has been fact-checked and approved by 
        the verifier agent, then respond with a completion message.""",
        tools=bing.definitions,
        headers={"x-ms-enable-preview": "true"},
    )
    console.print("🎯 Created Orchestrator Agent", style="green")

    return project_client, writer_agent, editor_agent, verifier_agent, orchestrator_agent

# Create our team of agents
project_client, writer_agent, editor_agent, verifier_agent, orchestrator_agent = create_journalism_research_agent()


In [ ]:
## Starting the Article Writing Process

Now that we have our team of agents ready, let's start writing an article! 

In this cell, we'll:
1. Create a new conversation thread
2. Set our article topic
3. Add today's date to the article

You can modify the `article_topic` variable below to write about any topic you're interested in!


In [ ]:
# Set your article topic here
article_topic = "The impact of artificial intelligence on modern journalism"

# Create a new thread for our conversation
thread = project_client.agents.create_thread()
console.print("📜 Created new conversation thread", style="green")

# Add today's date to the topic
today_date = datetime.datetime.now().strftime("%d %B %Y")
full_prompt = f"Article topic: {article_topic}\nDate: {today_date}"

# Create initial message
message = project_client.agents.create_message(
    thread_id=thread.id,
    role=MessageRole.USER,
    content=full_prompt,
)
console.print("✉️ Created initial message with topic", style="green")


In [ ]:
## The Writing Process

Now we'll watch our team of agents work together to create the article. Each agent will perform its role in sequence:

1. Writer creates the first draft
2. Editor reviews and provides feedback
3. Writer improves based on feedback
4. Verifier fact-checks the article
5. Writer makes final adjustments
6. Orchestrator evaluates the final result

Let's run each step and see how the article evolves!


In [ ]:
# Step 1: Writer creates first draft
console.print("\n[bold blue]Step 1: Writer creates first draft...[/bold blue]")
run = project_client.agents.create_and_process_run(thread_id=thread.id, agent_id=writer_agent.id)
print_agent_response(project_client, thread.id, "Writer")

# Step 2: Editor reviews
console.print("\n[bold blue]Step 2: Editor reviews the draft...[/bold blue]")
run = project_client.agents.create_and_process_run(thread_id=thread.id, agent_id=editor_agent.id)
print_agent_response(project_client, thread.id, "Editor")

# Step 3: Writer improves
console.print("\n[bold blue]Step 3: Writer improves the article...[/bold blue]")
run = project_client.agents.create_and_process_run(thread_id=thread.id, agent_id=writer_agent.id)
print_agent_response(project_client, thread.id, "Writer")

# Step 4: Verifier fact-checks
console.print("\n[bold blue]Step 4: Verifier fact-checks the article...[/bold blue]")
run = project_client.agents.create_and_process_run(thread_id=thread.id, agent_id=verifier_agent.id)
print_agent_response(project_client, thread.id, "Verifier")

# Step 5: Writer makes final adjustments
console.print("\n[bold blue]Step 5: Writer makes final adjustments...[/bold blue]")
run = project_client.agents.create_and_process_run(thread_id=thread.id, agent_id=writer_agent.id)
print_agent_response(project_client, thread.id, "Writer")

# Step 6: Orchestrator evaluates
console.print("\n[bold blue]Step 6: Orchestrator evaluates the final article...[/bold blue]")
run = project_client.agents.create_and_process_run(thread_id=thread.id, agent_id=orchestrator_agent.id)
print_agent_response(project_client, thread.id, "Orchestrator")


In [ ]:
## Cleanup

Finally, let's clean up our resources by removing the agents we created. This is good practice to avoid unnecessary resource usage.


In [ ]:
# Clean up by removing the agents
console.print("\n[bold blue]Cleaning up resources...[/bold blue]")
project_client.agents.delete_agent(writer_agent.id)
project_client.agents.delete_agent(editor_agent.id)
project_client.agents.delete_agent(verifier_agent.id)
project_client.agents.delete_agent(orchestrator_agent.id)
console.print("✨ All agents have been removed", style="green")


### Creating Our AI Agents

Now we'll create our team of AI agents. Each agent has:
- A specific role and purpose
- Custom instructions that guide their behavior
- Access to the Bing Search tool for research

Let's create them one by one and see what makes each special!